In [1]:
%matplotlib inline

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import re

import gymnasium as gym
from gymnasium import spaces

import time

In [4]:
pd.read_csv('data/raw/d_2024_07_01.dat', encoding='Windows-1252')

In [13]:
with open('data/raw/00444174.dat', 'r', encoding='Windows-1252') as f:
    data = f.readlines()

In [27]:
data[:5]

In [25]:
with open('data/raw/01BB6174.dat', 'r', encoding='Windows-1252') as f:
    data2 = f.readlines()

In [26]:
data2[1]

In [49]:
with open('data/raw/d_2024_07_01.dat', 'r', errors='replace') as f:
    data = []
    for i in range(100):
        data.append(f.readline())

In [62]:
data[10:20]

In [94]:
def is_valid_nmea(sentence):
    # Basic check to see if it's a valid NMEA sentence (starts with $ and has a checksum *)
    return sentence.startswith('$') and '*' in sentence


def reg_match(line):
    pattern = r'\$(.*?)\*'
    matches = re.findall(pattern, line)

    return matches

def extract_valid_nmea_lines(file_path):
    valid_lines = []
    with open(file_path, 'r', errors='replace') as f:
        for line in f:
            line = line.strip()  # Remove any leading/trailing spaces or newlines
            matches = reg_match(line)
            if matches:
                if len(valid_lines) > 0 and valid_lines[-1] == matches[0]:
                    continue
                if 'GPRMC' in matches[0]:
                    valid_lines.append(matches[0])
    return valid_lines

file_path = 'data/raw/d_2024_07_01.dat'
valid_nmea_lines = extract_valid_nmea_lines(file_path)

In [95]:
valid_nmea_lines[:200]

In [85]:
# Function to extract time and sensor data
def parse_data(row):
    # Regex to extract the time component from sensor data (GPRMC, GPGGA etc.)
    time_pattern = re.compile(r'\d{6}\.\d{2}')
    
    for entry in row:
        time_match = time_pattern.search(entry)
        if time_match:
            time_str = time_match.group()
            # Convert time to hh:mm:ss
            time = f"{time_str[:2]}:{time_str[2:4]}:{time_str[4:6]}"
            sensor_type, sensor_data = entry.split(',', 1)
            return time, sensor_type, sensor_data
    
    return None, None, None

# Initialize an empty dictionary for the table
sensor_table = {}

# Process each row of the data
for row in valid_nmea_lines:
    time, sensor_type, sensor_data = parse_data(row)
    
    if time and sensor_type:
        if time not in sensor_table:
            sensor_table[time] = {}
        # Add the sensor data to the appropriate time row
        sensor_table[time][sensor_type] = sensor_data

# Convert the dictionary to a pandas DataFrame
df = pd.DataFrame.from_dict(sensor_table, orient='index')

# Show the DataFrame
print(df)


In [70]:
with open('data/raw/00444174.dat', 'r', errors='replace') as f:
    data = []
    for i in range(100):
        data.append(f.readline())

In [82]:
data[:20]

In [83]:
data[20:40]

In [103]:
log = pd.read_csv('data/raw/LOG 30.CSV', encoding='Windows-1252')

In [104]:
log['Target Name'].value_counts()

In [105]:
log['Type'].value_counts()

In [110]:
log[log['Type'] == 'ENC Data']['Message']

In [271]:
log = pd.read_csv('data/raw/LOG current.CSV', encoding='Windows-1252')

In [272]:
log

In [273]:
log = log.drop(columns=['MMSI', 'Type', 'Target Name'])

In [274]:
log.columns = ['date', 'time', 'msg']

In [275]:
log.loc[0, 'msg']

In [276]:
log[log.msg == " Manual Position Fix Added:  09°34.091'N  086°30.701'W"]

In [277]:
'Manual' in log.msg.str.strip()

In [278]:
log = log[~log['msg'].str.contains('Manual', case=False, na=False)]

In [279]:
log = log.reset_index(drop=True)

In [281]:
line = log.msg[0]

In [282]:
line = line.split(', ')

In [283]:
for i in range(len(line)):
    print(f'{i} - {line[i]}')

In [284]:
lat = line[0].split(': ')[1]
lon = line[1]
sog = line[2].split(': ')[1].split(' ')[0]
cog = line[3].split(': ')[1][:-1]
hdg = line[4].split(': ')[1].strip()[:-1]
wnd_dir = line[5].split(': ')[1][:-1]
wnd_spd = line[6].split(' ')[0]

In [285]:
deg, min, dir = lat.split(' ')

In [286]:
deg = deg[:-1]

In [287]:
min = min[:-1]

In [288]:
min

In [289]:
float(deg)

In [344]:
def extract_data_from_msg(line):
    try:
        line = line.split(', ')
        
        lat = line[0].split(': ')[1]
        lon = line[1]
        sog = line[2].split(': ')[1].split(' ')[0]
        cog = line[3].split(': ')[1][:-1]
        hdg = line[4].split(': ')[1].strip()[:-1]
        wnd_dir = line[5].split(': ')[1][:-1]
        wnd_spd = line[6].split(' ')[0]
    
        lat = convert_coordinates(lat)
        lon = convert_coordinates(lon)
    except IndexError:
        print(line)
        return [None] * 7

    return lat, lon, sog, cog, hdg, wnd_dir, wnd_spd

def convert_coordinates(coordinates):
    deg, min, dir = coordinates.split(' ')
    deg = deg[:-1]
    min = min[:-1]

    if dir == 'N' or dir == 'E':
        dir = 1
    else:
        dir = -1
    
    return np.round((float(deg) + float(min) / 60) * dir, 4)    

def drop_rows(df, msgs=['Manual', 'System', 'Total']):
    for msg in msgs:
        df = df[~df['msg'].str.contains(msg, case=False, na=False)]
    return df

def drop_columns(df, columns=['MMSI', 'Type', 'Target Name']):
    return df.drop(columns=columns)

def rename_columns(df, names=['date', 'time', 'msg']):
    df.columns = names
    return df

def clean_ship_log(path):
    df = pd.read_csv(path, encoding='Windows-1252')
    df = drop_columns(df)
    df = rename_columns(df)
    df = drop_rows(df)
    df[['lat', 'lon', 'sog', 'cog', 'hdg', 'wnd_dir', 'wnd_spd']] = df.msg.apply(lambda x: pd.Series(extract_data_from_msg(x)))
    df.date = pd.to_datetime(df.date)
    df.time = pd.to_datetime(df.time, format='%H:%M:%S').dt.time
    df = df.reset_index(drop=True)
    df = drop_columns(df, 'msg')
    return df
    

In [291]:
line_2 = log.msg[1]

In [292]:
print(extract_data_from_msg(line_2))

In [299]:
log_50 = log[:50]

In [300]:
log_50.loc[:, ['lat', 'lon', 'sog', 'cog', 'hdg', 'wnd_dir', 'wnd_spd']] = log_50.msg.apply(lambda x: pd.Series(extract_data_from_msg(x)))

In [301]:
log.msg.apply(lambda x: pd.Series(extract_data_from_msg(x)))

In [304]:
log[log.msg.str.contains('System Turned On')]

In [ ]:
log[log.msg.str.contains('Total Distance Passed')]

In [305]:
log = log[~log['msg'].str.contains('Total Distance Passed', case=False, na=False)]

In [307]:
log = log[~log['msg'].str.contains('System Turned On', case=False, na=False)]

In [308]:
log.reset_index(inplace=True, drop=True)

In [309]:
log

In [322]:
log.loc[:, ['lat', 'lon', 'sog', 'cog', 'hdg', 'wnd_dir', 'wnd_spd']] = log.msg.apply(lambda x: pd.Series(extract_data_from_msg(x)))

In [313]:
log = log.drop(columns='msg')

In [314]:
log

In [317]:
log.date = pd.to_datetime(log.date)

In [318]:
pd.to_datetime(log.time)

In [319]:
log

In [324]:
log = pd.read_csv('data/raw/LOG current.CSV', encoding='Windows-1252')

In [345]:
df = clean_ship_log('data/raw/LOG current.CSV')

In [348]:
df

In [16]:
def mercator_sailing(lat1, lon1, lat2, lon2):
    # Convert degrees to radians
    lat1 = np.radians(lat1)
    lat2 = np.radians(lat2)
    lon1 = np.radians(lon1)
    lon2 = np.radians(lon2)
    
    # Mercator latitudes
    def mercator_latitude(lat):
        return np.log(np.tan(np.pi / 4 + lat / 2))
    
    phi_m1 = mercator_latitude(lat1)
    phi_m2 = mercator_latitude(lat2)
    
    # Difference in longitudes
    delta_lambda = lon2 - lon1
    
    # Calculate distance using the Mercator Sailing formula
    D = np.sqrt((delta_lambda * np.cos(lat1))**2 + (phi_m2 - phi_m1)**2) * 3440.065
    return D

def bearing_to_waypoint(lat1, lon1, lat2, lon2):
    # Convert degrees to radians
    lat1 = np.radians(lat1)
    lat2 = np.radians(lat2)
    lon1 = np.radians(lon1)
    lon2 = np.radians(lon2)
    
    # Difference in longitude
    delta_lambda = lon2 - lon1
    
    # Mercator latitudes
    phi1 = np.log(np.tan(np.pi / 4 + lat1 / 2))
    phi2 = np.log(np.tan(np.pi / 4 + lat2 / 2))
    
    # Calculate the bearing using atan2
    bearing_rad = np.arctan2(delta_lambda, (phi2 - phi1))
    
    # Convert from radians to degrees
    bearing_deg = np.degrees(bearing_rad)
    
    # Normalize to 0-360 degrees
    bearing_deg = (bearing_deg + 360) % 360
    
    return bearing_deg

def mercator_sailing_future_position(lat, lon, speed, bearing, time_interval):
    """
    Calculate future position given current lat, lon, speed (knots), bearing, and time interval (hours).
    
    Parameters:
    - lat (float): Current latitude in degrees
    - lon (float): Current longitude in degrees
    - speed (float): Speed in knots (nautical miles per hour)
    - bearing (float): Bearing in degrees (from north)
    - time_interval (float): Time interval in hours
    
    Returns:
    - new_lat (float): New latitude in degrees
    - new_lon (float): New longitude in degrees
    """
    # Convert degrees to radians
    lat = np.radians(lat)
    lon = np.radians(lon)
    bearing = np.radians(bearing)

    # Earth radius in nautical miles
    R = 3440.065
    
    # Distance traveled in nautical miles
    distance = speed * time_interval

    # Delta latitude (in radians)
    delta_lat = distance * np.cos(bearing) / R
    
    # Update latitude
    new_lat = lat + delta_lat
    
    # Mercator latitudes for loxodromic calculation
    def mercator_lat(lat):
        return np.log(np.tan(np.pi / 4 + lat / 2))

    phi1 = mercator_lat(lat)
    phi2 = mercator_lat(new_lat)
    
    # Delta longitude (in radians)
    if np.cos(new_lat) != 0:
        delta_lon = (distance * np.sin(bearing)) / (R * np.cos(new_lat))
    else:
        delta_lon = 0
    
    # Update longitude
    new_lon = lon + delta_lon
    
    # Convert radians back to degrees
    new_lat = np.degrees(new_lat)
    new_lon = np.degrees(new_lon)
    
    return np.array([new_lat, new_lon])

In [34]:
# Example usage:
lat1 = 20.0   # Latitude of current position
lon1 = 55.4  # Longitude of current position
lat2 = 0.0   # Latitude of waypoint
lon2 = 55.4  # Longitude of waypoint

distance = mercator_sailing(lat1, lon1, lat2, lon2)
print(f"Distance to waypoint: {distance:.2f} nautical miles")
bearing = bearing_to_waypoint(lat1, lon1, lat2, lon2)
print(f"Bearing to waypoint: {bearing:.2f} deg")
new_lat, new_lon = mercator_sailing_future_position(lat1, lon1, speed=14, bearing=180, time_interval=90)
print(f"New position: Latitude {new_lat:.5f}, Longitude {new_lon:.5f}")

In [59]:
class ShipEnv(gym.Env):
    def __init__(self, initial_position, waypoint):
        super(ShipEnv, self).__init__()
        self.action_space = spaces.Discrete(4)  # Simplified: 0 = turn left, 1 = turn right, 2 = speed up, 3 = slow down
        self.observation_space = spaces.Box(low=-np.inf, high=np.inf, shape=(4,), dtype=np.float32)  # Simplified state
        
        self.initial_position = np.array(initial_position)
        self.waypoint = np.array(waypoint)
        self.current_position = np.copy(self.initial_position)
        self.speed = 14.4  # Example starting speed
        self.heading = 295.0  # Example starting heading
        self.done = False

        # Define discretization bins for each state component
        self.lat_bins = np.linspace(9.0, 11.0, 10)  # Example latitude bins
        self.lon_bins = np.linspace(-87.0, -85.0, 10)  # Example longitude bins
        self.speed_bins = np.linspace(10, 20, 5)  # Example speed bins
        self.heading_bins = np.linspace(0, 360, 8)  # Example heading bins

    def discretize_state(self, state):
        lat, lon, speed, heading = state
        lat_idx = np.digitize(lat, self.lat_bins) - 1  # Bin index for latitude
        lon_idx = np.digitize(lon, self.lon_bins) - 1  # Bin index for longitude
        speed_idx = np.digitize(speed, self.speed_bins) - 1  # Bin index for speed
        heading_idx = np.digitize(heading, self.heading_bins) - 1  # Bin index for heading
        return lat_idx, lon_idx, speed_idx, heading_idx

    def step(self, action):
        # Perform action: adjust heading or speed
        if action == 0:  # Turn left
            self.heading -= 5
        elif action == 1:  # Turn right
            self.heading += 5
        elif action == 2:  # Speed up
            self.speed += 0.5
        elif action == 3:  # Slow down
            self.speed -= 0.5

        # Calculate new position based on speed and heading (simplified movement update)
        self.current_position = update_position(self.current_position, self.heading, self.speed)
        
        # Calculate the distance to the waypoint
        distance_to_waypoint = calculate_distance(self.current_position, self.waypoint)
        reward = -distance_to_waypoint  # The closer to the waypoint, the better

        # Check if we are close enough to the waypoint
        self.done = distance_to_waypoint < 0.01  # Done if close enough

        # Simplified state representation
        state = np.array([self.current_position[0], self.current_position[1], self.speed, self.heading])
        discrete_state = self.discretize_state(state)
        return discrete_state, reward, self.done, {}

    def reset(self):
        # Reset the environment for a new episode
        self.current_position = np.copy(self.initial_position)
        self.speed = 14.4
        self.heading = 295.0
        self.done = False
        state = np.array([self.current_position[0], self.current_position[1], self.speed, self.heading])
        return self.discretize_state(state)

def calculate_distance(current_position, waypoint):
    lat1, lon1 = current_position
    lat2, lon2 = waypoint
    
    return mercator_sailing(lat1, lon1, lat2, lon2)

def update_position(current_position, heading, speed):
    lat1, lon1 = current_position
    return mercator_sailing_future_position(lat1, lon1, speed=speed, bearing=heading, time_interval=1)

# Q-learning implementation
def q_learning(env, num_episodes=1000, alpha=0.1, gamma=0.99, epsilon=0.1):
    # Create Q-table with discrete state space (size based on bins)
    q_table = np.zeros((10, 10, 5, 8, env.action_space.n))  # Size of the discretized space + actions

    for episode in range(num_episodes):
        state = env.reset()
        done = False

        while not done:
            # Epsilon-greedy action selection
            if np.random.rand() < epsilon:
                action = env.action_space.sample()  # Random action
            else:
                action = np.argmax(q_table[state])  # Best action from Q-table

            # Take the action
            next_state, reward, done, _ = env.step(action)

            # Q-learning update
            best_next_action = np.argmax(q_table[next_state])
            q_table[state][action] += alpha * (reward + gamma * q_table[next_state][best_next_action] - q_table[state][action])

            state = next_state

        if episode % 100 == 0:
            print(f"Episode {episode}, Reward: {reward}")

    return q_table


In [ ]:
# Running the environment
env = ShipEnv([0.0, 0.0], [0.5, 0.0])  # Example starting position and waypoint
q_table = q_learning(env)

# Test the trained Q-table
state = env.reset()
done = False
total_reward = 0
while not done:
    action = np.argmax(q_table[state])
    print(action)
    state, reward, done, _ = env.step(action)
    print(state, reward, done)
    total_reward += reward

print(f"Total reward after training: {total_reward}")


In [51]:
env = ShipEnv([0.0, 0.0], [0.5, 0.0])

In [54]:
q_learning(env)

In [58]:
class ShipEnv(gym.Env):
    def __init__(self, initial_position, waypoint, speed, heading):
        super(ShipEnv, self).__init__()
        # Action space: [adjust_heading, adjust_speed]
        self.action_space = spaces.Box(low=-1, high=1, shape=(2,), dtype=np.float32)
        # State space: [lat, lon, speed, heading, distance_to_waypoint, bearing_to_waypoint]
        self.observation_space = spaces.Box(low=-np.inf, high=np.inf, shape=(6,), dtype=np.float32)
        self.current_position = np.array(initial_position)
        self.waypoint = np.array(waypoint)
        self.speed = speed 
        self.heading = heading

    def step(self, action):
        # Adjust heading and speed based on action
        self.heading += action[0] * 10  # Adjust heading
        self.speed += action[1] * 0.5  # Adjust speed

        # Calculate new position based on speed and heading (simple simulation)
        self.current_position = update_position(self.current_position, self.heading, self.speed)

        # Calculate distance and bearing to waypoint
        distance = calculate_distance(self.current_position, self.waypoint)
        bearing = calculate_bearing(self.current_position, self.waypoint)

        # Define reward: positive reward for reducing distance
        reward = -distance  # Reward is negative distance to the waypoint

        done = distance < 0.01  # Episode ends if the distance is below a threshold
        state = np.array([*self.current_position, self.speed, self.heading, distance, bearing])

        return state, reward, done, {}

    def reset(self):
        # Reset the environment for a new episode
        self.current_position = np.array([0.0, 0.0])  # Initial position
        self.speed = 14.4
        self.heading = 295.0
        distance = calculate_distance(self.current_position, self.waypoint)
        bearing = calculate_bearing(self.current_position, self.waypoint)
        return np.array([*self.current_position, self.speed, self.heading, distance, bearing])

def calculate_distance(current_position, waypoint):
    return mercator_sailing(*current_position, *waypoint)

def calculate_bearing(current_position, waypoint):
    return bearing_to_waypoint(*current_position, *waypoint)

def update_position(current_position, heading, speed):
    return mercator_sailing_future_position(*current_position, heading, speed, time_interval=1/60)

In [94]:
def mercator_latitude(lat):
    return np.log(np.tan(np.pi / 4 + lat / 2))


def mercator_conversion(lat1, lon1, lat2, lon2):
    # Convert degrees to radians
    lat1 = np.radians(lat1)
    lat2 = np.radians(lat2)
    lon1 = np.radians(lon1)
    lon2 = np.radians(lon2)

    delta_phi = mercator_latitude(lat2) - mercator_latitude(lat1)

    # Difference in longitudes
    delta_lambda = lon2 - lon1

    return delta_phi, delta_lambda


def rumbline_distance(lat1, lon1, lat2, lon2):
    delta_phi, delta_lambda = mercator_conversion(lat1, lon1, lat2, lon2)

    # Calculate distance using the Mercator Sailing formula
    return np.sqrt((delta_lambda * np.cos(np.radians(lat1))) ** 2 + delta_phi ** 2) * 3440.065


def bearing_to_waypoint(lat1, lon1, lat2, lon2):
    delta_phi, delta_lambda = mercator_conversion(lat1, lon1, lat2, lon2)

    # Calculate the bearing using atan2
    bearing_rad = np.arctan2(delta_lambda, delta_phi)

    # Convert from radians to degrees
    bearing_deg = np.degrees(bearing_rad)

    # Normalize to 0-360 degrees
    bearing_deg = (bearing_deg + 360) % 360

    return bearing_deg


def mercator_sailing_future_position(lat, lon, speed, bearing, time_interval):
    """
    Calculate future position given current lat, lon, speed (knots), bearing, and time interval (hours).

    Parameters:
    - lat (float): Current latitude in degrees
    - lon (float): Current longitude in degrees
    - speed (float): Speed in knots (nautical miles per hour)
    - bearing (float): Bearing in degrees (from north)
    - time_interval (float): Time interval in hours

    Returns:
    - new_lat (float): New latitude in degrees
    - new_lon (float): New longitude in degrees
    """
    # Convert degrees to radians
    lat = np.radians(lat)
    lon = np.radians(lon)
    bearing = np.radians(bearing)

    # Earth radius in nautical miles
    R = 3440.065

    # Distance traveled in nautical miles
    distance = speed * time_interval

    # Delta latitude (in radians)
    delta_lat = distance * np.cos(bearing) / R

    # Update latitude
    new_lat = lat + delta_lat

    # Delta longitude (in radians)
    if np.cos(new_lat) != 0:
        delta_lon = (distance * np.sin(bearing)) / (R * np.cos(new_lat))
    else:
        delta_lon = 0

    # Update longitude
    new_lon = lon + delta_lon

    # Convert radians back to degrees
    new_lat = np.degrees(new_lat)
    new_lon = np.degrees(new_lon)

    return np.array([new_lat, new_lon])

class ShipEnv(gym.Env):
    def __init__(self, start_wp, tgt_wp, initial_position, initial_heading):
        super(ShipEnv, self).__init__()
        
        # Action space: discrete actions to adjust heading (left or right)
        self.action_space = spaces.Discrete(3)  # 0 = turn left, 1 = maintain heading, 2 = turn right
        
        # Observation space: [heading, bearing_to_target, distance_to_target]
        self.observation_space = spaces.Box(low=-np.inf, high=np.inf, shape=(3,), dtype=np.float32)
        
        self.start_wp = np.array(start_wp)
        self.tgt_wp = np.array(tgt_wp)
        self.current_position = np.array(initial_position)
        self.heading = initial_heading
        
        # Bearing and distance to the target waypoint
        self.bearing_to_target = self.calculate_bearing(self.current_position, self.tgt_wp)
        self.distance_to_target = self.calculate_distance(self.current_position, self.tgt_wp)
        self.initial_course = self.calculate_bearing(self.start_wp, self.tgt_wp)
        
    def step(self, action):
        
        if np.abs(self.heading - self.initial_course) >= 20:
            value = 10
        elif np.abs(self.heading - self.initial_course) >= 5:
            value = 5
        else:
            value = 1
        
        # Adjust the heading based on the action
        if action == 0:
            self.heading -= value  # Turn left by 1 degree
        elif action == 2:
            self.heading += value  # Turn right by 1 degree

        if self.heading < 0:
            self.heading += 360
        
        # Update the ASV position based on the current heading (this is simplified)
        self.current_position = self.update_position(self.current_position, self.heading)
        
        # Recalculate the distance and bearing to the target
        self.bearing_to_target = self.calculate_bearing(self.current_position, self.tgt_wp)
        self.distance_to_target = self.calculate_distance(self.current_position, self.tgt_wp)
        
        # Reward is based on reducing the distance and keeping the course close to the initial course
        bearing_diff = abs(self.bearing_to_target - self.initial_course)
        reward = -self.distance_to_target - bearing_diff
        
        # Done if the ASV is very close to the target waypoint
        done = self.distance_to_target < 0.01
        
        # State: [heading, bearing_to_target, distance_to_target]
        state = np.array([self.heading, self.bearing_to_target, self.distance_to_target, self.current_position[0], self.current_position[1]])
        
        return state, reward, done, {}
    
    def reset(self, initial_position, initial_heading):
        # Reset the environment for a new episode
        self.current_position = np.array(initial_position)  # Reset to starting waypoint
        self.heading = initial_heading  # Reset heading to the course between start_wp and tgt_wp
        
        # Recalculate distance and bearing
        self.bearing_to_target = self.calculate_bearing(self.current_position, self.tgt_wp)
        self.distance_to_target = self.calculate_distance(self.current_position, self.tgt_wp)
        
        # State: [heading, bearing_to_target, distance_to_target]
        return np.array([self.heading, self.bearing_to_target, self.distance_to_target, self.current_position[0], self.current_position[1]])
    
    def calculate_distance(self, current_position, target_position):
        return mercator_sailing(*current_position, *target_position)
    
    def calculate_bearing(self, current_position, target_position):
        return bearing_to_waypoint(*current_position, *target_position)
    
    def update_position(self, position, heading, speed=14, time_interval=1/60):
        # Update position based on heading (simplified movement for demonstration)
        # time.sleep(1)
        lat, lon = position
        # heading_rad = np.radians(heading)
        
        # delta_lat = speed * time_interval * np.cos(heading_rad)
        # delta_lon = speed * time_interval * np.sin(heading_rad)
        
        # return np.array([lat + delta_lat, lon + delta_lon])
        return mercator_sailing_future_position(lat, lon, speed, heading, time_interval)

# Define the Q-Learning Algorithm
def deterministic_policy(env):
    state = env.reset()
    done = False
    total_reward = 0

    while not done:
        # Get the current heading, bearing to target, and distance to target
        heading, bearing_to_target, distance_to_target = state

        # Determine the action based on the difference between current bearing and initial course
        if bearing_to_target < env.initial_course - 1:
            action = 0  # Turn left
        elif bearing_to_target > env.initial_course + 1:
            action = 2  # Turn right
        else:
            action = 1  # Maintain heading

        # Take the action in the environment
        next_state, reward, done, _ = env.step(action)
        total_reward += reward
        state = next_state

    return total_reward

In [98]:
start_wp = [0, 0]
tgt_wp = [0.2, 0.2]
asv_psn = [0.00, 0.05]
asv_hdg = 340
env = ShipEnv(start_wp, tgt_wp, asv_psn, asv_hdg)

# Run the deterministic policy
state = env.reset(asv_psn, asv_hdg)
done = False
total_reward = 0
plt.plot([start_wp[1], tgt_wp[1]], [start_wp[0], tgt_wp[0]])
# while not done:
for i in range(100):
    # Get the current heading, bearing to target, and distance to target
    heading, bearing_to_target, distance_to_target, lat, lon = state
    # Determine the action based on the difference between bearing to tgt and heading
    
    if bearing_to_target < heading - 1:
        action = 0  # Turn left
    elif bearing_to_target > heading + 1:
        action = 2  # Turn right
    else:
        action = 1  # Maintain heading
    # Take the action in the environment
    next_state, reward, done, _ = env.step(action)
    total_reward += reward
    state = next_state
    plt.scatter(lon, lat)
    # print(f"State: {state}, Reward: {reward}, Done: {done}")
    # time.sleep(1)
plt.show()
print(f"Total reward after following the policy: {total_reward}")

In [31]:
# Example usage
start_wp = [0, 0]
tgt_wp = [0.2, -0.1]
asv_psn = [0.01, -0.01]
asv_hdg = 300

env = ShipEnv(start_wp, tgt_wp, asv_psn, asv_hdg)

state = env.reset()
done = False

while not done:
    action = env.action_space.sample()  # Random action for testing
    state, reward, done, _ = env.step(action)
    print(f"State: {state}, Reward: {reward}, Done: {done}")

In [91]:
np.sin(10)

In [21]:
2.19e-01